# 범주형 데이터 이진분류 경진대회 베이스라인 모델

- [범주형 데이터 이진분류 경진대회 링크](https://www.kaggle.com/c/cat-in-the-dat/)

In [1]:
import pandas as pd

train = pd.read_csv('/kaggle/input/cat-in-the-dat/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/cat-in-the-dat/test.csv', index_col='id')
submission = pd.read_csv('/kaggle/input/cat-in-the-dat/sample_submission.csv', index_col='id')

### 훈련 데이터 테스트 데이터 합치기

In [2]:
# 훈련 데이터와 테스트 데이터 합치기 
all_data = pd.concat([train, test])
all_data = all_data.drop('target', axis=1) # 타깃 값 제거
all_data

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0,0,0,F,N,Green,Square,Lion,Canada,Theremin,...,9e4b23160,acc31291f,1,Novice,Lava Hot,j,A,Gb,1,3
499996,1,0,0,F,Y,Green,Trapezoid,Lion,China,Piano,...,cfbd87ed0,eae3446d0,1,Contributor,Lava Hot,f,S,Ed,2,2
499997,0,1,1,T,Y,Green,Trapezoid,Lion,Canada,Oboe,...,1108bcd6c,33dd3cf4b,1,Novice,Boiling Hot,g,V,TR,3,1


### 원-핫 인코딩 적용

In [3]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder() # 원-핫 인코더 객체 생성
all_data_encoded = encoder.fit_transform(all_data) # 원-핫 인코딩 적용

### 훈련 데이터, 테스트 데이터 나누기

In [4]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data_encoded[:num_train]
X_test = all_data_encoded[num_train:]

y_train = train['target']

### 로지스틱 회귀 모델 생성 및 훈련

In [5]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter=1000, random_state=42) # 모델 생성
logistic_model.fit(X_train, y_train) # 모델 훈련

LogisticRegression(max_iter=1000, random_state=42)

In [6]:
logistic_model.predict_proba(X_test)

array([[0.70348797, 0.29651203],
       [0.30860638, 0.69139362],
       [0.9274226 , 0.0725774 ],
       ...,
       [0.60813573, 0.39186427],
       [0.44702685, 0.55297315],
       [0.7879414 , 0.2120586 ]])

In [7]:
logistic_model.predict(X_test)

array([0, 1, 0, ..., 0, 1, 0])

### 예측

In [8]:
# 타깃 값 1일 확률 예측
y_preds = logistic_model.predict_proba(X_test)[:,1]

### 제출

In [9]:
# 제출 파일 생성
submission['target'] = y_preds
submission.to_csv('submission.csv')